In [1]:
import requests
import pandas as pd
import time
from datetime import datetime

BASE_URL = "https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"

def fetch_historical_prices(coin_id, vs_currency="inr", days=365):
    url = BASE_URL.format(coin_id=coin_id)

    params = {
        "vs_currency": vs_currency,
        "days": days
    }

    response = requests.get(url, params=params)

    if response.status_code == 429:
        print(f"Rate limit hit for {coin_id}. Sleeping...")
        time.sleep(10)
        return fetch_historical_prices(coin_id, vs_currency, days)

    response.raise_for_status()
    data = response.json()

    records = []
    for price_point in data["prices"]:
        timestamp, price = price_point
        date = datetime.utcfromtimestamp(timestamp / 1000).date()

        records.append({
            "coin_id": coin_id,
            "date": date,
            "price_usd": price   # price is actually INR
        })

    return records

In [5]:
df_crypto = pd.read_csv("D:\\Python_Code\\Project1-Cross-MarketAnalysisCryptoOil&StocksWithSQL&Streamlit\\coingecko_crypto_metadata.csv")

In [6]:
# df_crypto -> from coins/markets API
top3_coins = (
    df_crypto
    .sort_values("market_cap_rank")
    .head(3)["id"]
    .tolist()
)

print(top3_coins)

['bitcoin', 'ethereum', 'tether']


In [7]:
all_prices = []

for coin in top3_coins:
    print(f"Fetching historical prices for {coin}...")
    coin_data = fetch_historical_prices(coin)
    all_prices.extend(coin_data)
    time.sleep(2)  # avoid rate limiting

df_prices = pd.DataFrame(all_prices)

Fetching historical prices for bitcoin...


C:\Users\bless\AppData\Local\Temp\ipykernel_31708\3733984708.py:29: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.utcfromtimestamp(timestamp / 1000).date()


Fetching historical prices for ethereum...
Fetching historical prices for tether...


In [8]:
print(df_prices.head())
print(df_prices.tail())
print("Total records:", len(df_prices))

   coin_id        date     price_usd
0  bitcoin  2025-01-28  8.813005e+06
1  bitcoin  2025-01-29  8.773589e+06
2  bitcoin  2025-01-30  8.986343e+06
3  bitcoin  2025-01-31  9.079031e+06
4  bitcoin  2025-02-01  8.876282e+06
     coin_id        date  price_usd
1093  tether  2026-01-24  91.494639
1094  tether  2026-01-25  91.460823
1095  tether  2026-01-26  91.550111
1096  tether  2026-01-27  91.617287
1097  tether  2026-01-27  91.503664
Total records: 1098


In [9]:
df_prices.to_csv("D:\\Python_Code\\Project - 1 -  Cross-Market Analysis Crypto, Oil & Stocks with SQL and Streamlit\\top3_coins_historical_prices.csv", index=False)
